In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import scipy.io.wavfile as wav
import numpy as np
from python_speech_features import mfcc

# Generación de ficheros

In [2]:
import numpy as np
import os
def generate_file_data(dir,name):
    directory=dir
    a={'0':'zero','1':'one','2':'two','3':'three','4':'four','5':'five','6':'six','7':'seven','8':'eight','9':'nine'}
    da=os.listdir(directory)
    da.sort()
    file = open(dir+name+'.txt',"w")
    for filename in da:
        if '.wav' in filename:
            file.write(filename+','+a[filename[0]]+'\n')
    file.close() 
    with open(directory+'/'+name+'.txt') as f:
        read_data = f.read()
        f.closed
    read_data=read_data.split('\n')
    read_data=read_data[0:len(read_data)-1]
    return read_data

# Encoding words with One Hot Encoding

In [3]:
vocabulary_words=['zero','one','two','three','four','five','six','seven','eight','nine']
label_encoder=LabelEncoder()
integer_encoded = label_encoder.fit_transform(vocabulary_words)
print(integer_encoded)
onehot_encoder = OneHotEncoder(sparse=False,categories='auto')
integer_encoded = integer_encoded.reshape(-1, 1)
onehot_encoded=onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)

[9 4 8 7 2 1 6 5 0 3]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]


In [4]:
def encode(x):# will transform the label using one hot encoding/take a group of strings

    integers=label_encoder.transform(x)
    integers=integers.reshape(-1,1)
    return onehot_encoder.transform(integers)

In [5]:
x=encode(['zero','one'])
print(x)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [6]:
def decode(x):
    inverted = onehot_encoder.inverse_transform(x)
    y=inverted.astype(dtype=int)# castea los valores
    y=y.reshape(1,-1)
    y=y[0]
    y=label_encoder.inverse_transform(y)
    return y

In [7]:
y=decode(x)
print(y)

['zero' 'one']


# MFCC 


In [44]:
def mfcc_features(DIR,list_dir):
    mfcc_audios=[]
    for dir in list_dir:
        fs, audio = wav.read(DIR+dir)
        features = mfcc(audio, samplerate=fs)
        features=np.pad(features,((0,100-len(features)),(0,0)),mode='constant', constant_values=0)
        mfcc_audios.append(features)
    mfcc_audios=np.array(mfcc_audios)
    return mfcc_audios

In [45]:
def prepare_data(dir,name):
    file = open(dir+name)
    f=file.read()
    file.close()
    f=f.split('\n')
    f=f[0:len(f)-1]
    labels=[]
    names_audios=[]
    for i in f:
        j=i.split(',')
        names_audios.append(j[0])
        labels.append(j[1])
    labels=np.array(labels)
    onehot_encoded= encode(labels)
    #print(len(onehot_encoded))
    mfcc=mfcc_features(dir,names_audios)
    #print(onehot_encoded)
    #d=decode(onehot_encoded)
    #print(d)
    print(name+' OK')
    return mfcc,onehot_encoded



In [46]:
class dataset:
    def __init__(self,data):
        self.i=0#para el shuffle
        self.data_dir=data
        self.shuffle=None
        self.dir_training=data+'/training/'
        self.dir_test=data+'/test/'
        self.training_set=None
        self.test_set=None
    def split_dataset(self):
        generate_file_data(self.dir_training,name='training')
        generate_file_data(self.dir_test,name='test')
        self.training_set=prepare_data(self.dir_training,'training.txt')
        self.test_set=prepare_data(self.dir_test,'test.txt')
    #def next_batch(self,size):
    #
    #    return self.training_set[self.shuffle]
    

In [47]:
d=dataset('data')
d.split_dataset()

training.txt OK
test.txt OK


In [48]:
d.dir_test[0]

'd'

In [51]:
d.training_set[0][2].shape

(100, 13)

In [52]:
[d.training_set[0][i].shape[0] for i in range(80)]

[100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100]

In [53]:
d.training_set[0][76].shape

(100, 13)

In [54]:
d.test_set

(array([[[ 12.15175691, -34.77212437,   6.29363515, ...,  -1.66450868,
           -4.59147693,   1.46285289],
         [ 12.31191793, -30.53035182,  11.94208697, ...,  -1.92408134,
           -3.67758637, -11.9930407 ],
         [ 12.86984676, -34.34894095,   9.30594105, ..., -12.05547233,
          -12.88466489, -10.50400939],
         ...,
         [  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.        ],
         [  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.        ],
         [  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.        ]],
 
        [[  9.90856661,   1.12535726,  12.2697841 , ..., -11.32702006,
          -26.11778919,  -7.02361402],
         [ 10.44328459,   2.6602572 ,  18.27103714, ...,  -9.69676581,
          -21.74878267,   2.01730651],
         [ 10.96968797,   2.522129  ,  20.16315113, ..., -10.46480124,
          -24.84007463,   1.7928